# Suche Magnetfeld für optimale Korrektur mittels CMA-ES

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import cma
from STEP import STEP

In [ ]:
ebins = np.array([  0.98 ,   2.144,   2.336,   2.544,   2.784,   3.04 ,   3.312,
         3.6  ,   3.92 ,   4.288,   4.672,   5.088,   5.568,   6.08 ,
         6.624,   7.2  ,   7.84 ,   8.576,   9.344,  10.176,  11.136,
        12.16 ,  13.248,  14.4  ,  15.68 ,  17.152,  18.688,  20.352,
        22.272,  24.32 ,  26.496,  28.8  ,  31.36 ,  34.304,  37.376,
        40.704,  44.544,  48.64 ,  52.992,  57.6  ,  62.72 ,  68.608,
        74.752,  81.408,  89.088,  97.28 , 105.984, 115.2  , 125.44 ,
       137.216, 149.504, 162.816, 178.176, 194.56 , 211.968, 230.4  ,
       372.736])

def grenz(t):
    return -0.5*t + 20

# dat = STEP(2021, 12, 4, rpath='data/STEP/', mag_path='data/mag/srf', mag_frame = 'srf')
dat = STEP(2021,12,4,mag_path='default',mag_frame='srf')
period =(dt.datetime(2021,12,4,13,50),dt.datetime(2021,12,4,14,30))

STEP-Data loaded successfully.
STEP-Data combined successfully.
2021-12-04 00:00:00 /data/projects/solo/mag/l2_soar/srf/2021/solo_L2_mag-srf-normal_20211204_V01.cdf


In [3]:
def pw(flow,B,B_offset):
    '''Übergebe den particle flow-Vektor als Geschwindigkeit und den Magnetfeldvektor (am besten in SRF) und berechne die Pitchwinkel über das Skalarprodukt.
    Zusätzlich kann für die Magnetfeldkomponenten ein konstanter Offset übergeben werden.'''
    len_flow = np.sqrt(flow[0]**2 + flow[1]**2 + flow[2]**2)
    len_B = np.sqrt((B[0]+B_offset[0])**2 + (B[1]+B_offset[1])**2 + (B[2]+B_offset[2])**2)
    argument = (flow[0]*(B[0]+B_offset[0]) + flow[1]*(B[1]+B_offset[1]) + flow[2]*(B[2]+B_offset[2]))/len_flow/len_B
    result = np.arccos(argument)
    return result
        
def calc_pw(dat,B_offset):
    '''Berechne die Pitchwinkel für die Elektronen, welche auf STEP treffen in erster Näherung.
    Dafür wird der Winkel zwischen dem particle flow vector der Pixel und dem Magnetfeld herangezogen.
    Kann wieder einen Offset für das Magnetfeld übergeben.'''
    pws =  []
    for i in range(15):
        pws.append(pw(dat.flow_vector[i],[dat.B_R,dat.B_T,dat.B_N],B_offset))
    return np.array(pws)

def average_pw(dat,period,pitchangles,window_width=5):
    '''Berechnung der gemittelten Pitchwinkel'''
    # Maske, da ich nur die Magnetfelddaten innerhalb von period brauche:
    mask = (dat.time > period[0]) * (dat.time <= period[1])
    pw = [[] for i in range(15)]
    pw_time = []
        
    i = 0
    while (period[0] + dt.timedelta(minutes=(i+1)*window_width)) <= period[1]:
        pw_time.append(period[0] + dt.timedelta(minutes=(i+0.5)*window_width))
            
        for k in [i for i in range(1,16)]:
            # Mittelung der Pitchwinkel (k-1, da ich keine Zeit im array stehen habe)
            pw_data = pitchangles[k-1][mask]
            new_pw = np.sum(pw_data[i*window_width:(i+1)*window_width])/window_width
            pw[k-1].append(new_pw)
        i +=1
    return pw, pw_time

def step_plot_correction_multiple_offsets(dat, period, grenz, Offsets, title=None):
    '''Berechne und Plotte die Korrektur für manipulierte Magnetfelddaten und packe alles in einen Plot.'''

    pixel_means, pixel_var = dat.calc_energy_means(ebins=ebins,head=-1, period=period, grenzfunktion=grenz, norm='ptmax')
    
    # Offsets = np.array([np.zeros(11),np.zeros(11),np.array([i for i in range(-5,6)])]).T

    pixel1 = 3
    
    fig, ax = dat.step_plot('time', 'difference of energy means [keV]', f'difference of corrected energy means to pixel {pixel1}')

    for B_offset in Offsets:
        pitchangles = calc_pw(dat.mag,B_offset)
        pw, pw_time = average_pw(dat.mag,period,pitchangles)

        year = str(period[0].year - 2000)
        if period[0].month < 10:
            month = '0' + str(period[0].month)
        else:
            month = str(period[0].month)
        if period[0].day < 10:
            day = '0' + str(period[0].day)
        else:
            day = str(period[0].day)

        ax[0].plot([],[],label=str(B_offset))

        for pixel2 in range(1,16):
            # Übergebe willkürliche Fehler, da ich diese eh nicht brauche.
            energy2_corrected = dat.energy_correction(pixel_means[pixel2],pw[pixel1-1],pw[pixel2-1],2,2)[0]
            diff_corrected = energy2_corrected - pixel_means[pixel1]
            
            ax[pixel2].plot(pixel_means[0],diff_corrected,marker='x')
            ax[pixel2].axhline(0,color='tab:red')
                
            ax[pixel2].tick_params(axis='x',labelrotation=45)
    ax[0].legend()
    if type(title) == str:
        plt.savefig(f'mag_variation_cmaes/' + title + '.png')
    else:
        plt.savefig(f'mag_variation_cmaes/step_plot_total_correction_differences_energy_pixel{pixel1}_{year}_{month}_{day}_multiple_offsets.png')
    plt.close('all')

In [4]:
def func_to_min(B_offset,pixel_means,reference_pixel=3):
        '''Berechne für jeden Pixel den Mittelwert der Abweichungen aus allen Werten der Zeitreihe.
        Anschließend mittele ich über alle Pixel.'''
        deviation_of_pixels = []

        for pixel in range(1,16):
                pitchangles = calc_pw(dat.mag,B_offset)
                pw, pw_time = average_pw(dat.mag,period,pitchangles)

                energy2_corrected = dat.energy_correction(pixel_means[pixel],pw[reference_pixel-1],pw[pixel-1],2,2)[0]
                diff_corrected = energy2_corrected - pixel_means[reference_pixel]
                pixel_deviation = np.sum(diff_corrected)/len(diff_corrected)
                deviation_of_pixels.append(pixel_deviation)

        total_deviation = np.sum(np.array(deviation_of_pixels))/len(deviation_of_pixels)
        print('calculated function to minimize')
        return abs(total_deviation)

def func_to_min2(B_offset,pixel_means,reference_pixel=3):
        '''Berechne für jeden Pixel den Mittelwert der Abweichungen aus allen Werten der Zeitreihe.
        Anschließend mittele ich über alle Pixel.'''
        deviation_of_pixels = np.array([])

        for pixel in range(1,16):
                pitchangles = calc_pw(dat.mag,B_offset)
                pw, pw_time = average_pw(dat.mag,period,pitchangles)

                energy2_corrected = dat.energy_correction(pixel_means[pixel],pw[reference_pixel-1],pw[pixel-1],2,2)[0]
                diff_corrected = energy2_corrected - pixel_means[reference_pixel]
                deviation_of_pixels = np.concatenate((deviation_of_pixels,diff_corrected))

        total_deviation = np.sum(np.array(deviation_of_pixels))/len(deviation_of_pixels)
        print('calculated function to minimize')
        return abs(total_deviation)

In [5]:
pixel_means = dat.calc_energy_means(ebins=ebins,head=-1, period=period, grenzfunktion=grenz, norm='ptmax')[0]

In [28]:
B_offset0 = [0,0,0]

x_opt, es = cma.fmin2(objective_function = func_to_min2, x0 = [B_offset0], sigma0 = 0.5, args = [pixel_means], options = {'maxfevals':100})
print(x_opt)

c:\Users\ebel1\anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: input x0 should be a list or 1-D array, trying to flatten (1, 3)-array ()
  warnings.warn(msg + ' (' +


(3_w,7)-aCMA-ES (mu_w=2.3,w_1=58%) in dimension 3 (seed=897942, Fri Sep 15 13:11:02 2023)
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      7 3.578349503817720e-02 1.0e+00 4.28e-01  4e-01  4e-01 1:03.7
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
    2     14 2.385157695454521e-02 1.3e+00 5.60e-01  5e-01  7e-01 2:07.0
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
    3     21 1.43249

In [29]:
es.result_pretty()

termination on maxfevals=100
final/bestever f-value = 8.853263e-04 8.853263e-04 after 106/87 evaluations
incumbent solution: [-1.1598118207096957, 0.26771628503645917, -1.2669826122156809]
std deviation: [0.3994998164282894, 0.19171449257604734, 0.2988988485369194]


CMAEvolutionStrategyResult(xbest=array([-1.46507209,  0.41276429, -1.39693563]), fbest=0.0008853263063872536, evals_best=87, evaluations=106, iterations=15, xfavorite=array([-1.15981182,  0.26771629, -1.26698261]), stds=array([0.39949982, 0.19171449, 0.29889885]), stop={'maxfevals': 100})

In [30]:
B_offset = x_opt
print(B_offset)

step_plot_correction_multiple_offsets(dat,period,grenz,[[0,0,0],B_offset],'mag_variation_cmaes_2021_12_04_second_try')

[-1.46507209  0.41276429 -1.39693563]


Versuche Pixel einzeln zu optimieren und den Magnetfeld-Offset für die geringste Abweichung zum Referenzpixe zu finden:

In [8]:
def func_to_min_single_pixel(B_offset,pixel_means,pixel,reference_pixel=3):
    '''Berechne Abstand der korrigierten Energie zum Referenzpixel für einen einzelnen Pixel.'''
    pitchangles = calc_pw(dat.mag,B_offset)
    pw, pw_time = average_pw(dat.mag,period,pitchangles)

    energy2_corrected = dat.energy_correction(pixel_means[pixel],pw[reference_pixel-1],pw[pixel-1],2,2)[0]
    diff_corrected = energy2_corrected - pixel_means[reference_pixel]
    pixel_deviation = np.sum(diff_corrected)/len(diff_corrected)
    print('calculated function to minimize')
    return abs(pixel_deviation)

In [10]:
B_offset0 = [0,0,0]
B_offsets_pixel = []
es_pixel = []

for pixel in range(1,16):
    x_opt, es = cma.fmin2(objective_function = func_to_min_single_pixel, x0 = [B_offset0], sigma0 = 0.5, args = [pixel_means,pixel], options = {'maxfevals':100})
    B_offsets_pixel.append(x_opt)
    es_pixel.append(es)

print('Idealer Magnetfeld-Offset:')
for pixel in range(1,16):
    print(f'Pixel {pixel}: {B_offsets_pixel[pixel-1]}')

(3_w,7)-aCMA-ES (mu_w=2.3,w_1=58%) in dimension 3 (seed=103078, Mon Sep 18 09:38:07 2023)
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      7 1.341813968027975e+00 1.0e+00 4.31e-01  4e-01  4e-01 0:02.7
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
    2     14 6.653627835839511e-01 1.3e+00 4.34e-01  4e-01  4e-01 0:05.3
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
calculated function to minimize
    3     21 3.20305

In [13]:
def step_plot_different_offsets_each_pixel(dat, period, grenz, Offsets, title=None):
    '''Übergebe für jedem Pixel einen eigenen Magnetfeld-Offset, um ideale Korrektur zu erreichen.'''

    pixel_means, pixel_var = dat.calc_energy_means(ebins=ebins,head=-1, period=period, grenzfunktion=grenz, norm='ptmax')
    
    # Offsets = np.array([np.zeros(11),np.zeros(11),np.array([i for i in range(-5,6)])]).T

    pixel1 = 3
    
    fig, ax = dat.step_plot('time', 'difference of energy means [keV]', f'difference of corrected energy means to pixel {pixel1}')

    year = str(period[0].year - 2000)
    if period[0].month < 10:
         month = '0' + str(period[0].month)
    else:
        month = str(period[0].month)
    if period[0].day < 10:
        day = '0' + str(period[0].day)
    else:
        day = str(period[0].day)

    for pixel2 in range(1,16):
        pitchangles = calc_pw(dat.mag,Offsets[pixel2-1])
        pw, pw_time = average_pw(dat.mag,period,pitchangles)
            
        # Übergebe willkürliche Fehler, da ich diese eh nicht brauche.
        energy2_corrected = dat.energy_correction(pixel_means[pixel2],pw[pixel1-1],pw[pixel2-1],2,2)[0]
        diff_corrected = energy2_corrected - pixel_means[pixel1]
            
        ax[pixel2].plot(pixel_means[0],diff_corrected,marker='x',label=f'{Offsets[pixel2-1]}')
        ax[pixel2].axhline(0,color='tab:red')
                
        ax[pixel2].tick_params(axis='x',labelrotation=45)
        ax[pixel2].legend()
    if type(title) == str:
        plt.savefig(f'mag_variation_cmaes/' + title + '.png')
    else:
        plt.savefig(f'mag_variation_cmaes/step_plot_total_correction_differences_energy_pixel{pixel1}_{year}_{month}_{day}_multiple_offsets.png')
    plt.close('all')

In [14]:
step_plot_different_offsets_each_pixel(dat, period, grenz, B_offsets_pixel, title='optimal_correction_each_pixel')

Berechne ideales Magnetfeld für jeden Zeitpunkt meiner Zeitreihe (mit externem Skript):

In [ ]:
def step_plot_ideal_offsets_each_ts(dat, period, grenz, Offsets_ts, title=None):
    '''Übergebe Zeitreihe von idealen Magnetfeld-Offsets, um damit ideale Korrektur zu berechnen.'''

    pixel_means, pixel_var = dat.calc_energy_means(ebins=ebins,head=-1, period=period, grenzfunktion=grenz, norm='ptmax')
    
    len_ts = len(pixel_means[0])
    
    # Offsets = np.array([np.zeros(11),np.zeros(11),np.array([i for i in range(-5,6)])]).T

    pixel1 = 3
    
    fig, ax = dat.step_plot('time', 'difference of energy means [keV]', f'difference of corrected energy means to pixel {pixel1}')

    year = str(period[0].year - 2000)
    if period[0].month < 10:
         month = '0' + str(period[0].month)
    else:
        month = str(period[0].month)
    if period[0].day < 10:
        day = '0' + str(period[0].day)
    else:
        day = str(period[0].day)

    for pixel2 in range(1,16):
        diff_of_pixel = []
        for pot in range(len_ts):
            # Gehen die folgenden zwei zeilen einfacher?
            pitchangles = calc_pw(dat.mag,Offsets_ts[pot])
            pw, pw_time = average_pw(dat.mag,period,pitchangles)
            
            # Übergebe willkürliche Fehler, da ich diese eh nicht brauche.
            energy2_corrected = dat.energy_correction(pixel_means[pixel2][pot],pw[pixel1-1][pot],pw[pixel2-1][pot],2,2)[0]
            diff_corrected = energy2_corrected - pixel_means[pixel1][pot]
            diff_of_pixel.append(diff_corrected)
            
        ax[pixel2].plot(pixel_means[0],diff_of_pixels,marker='x')
        ax[pixel2].axhline(0,color='tab:red')
                
        ax[pixel2].tick_params(axis='x',labelrotation=45)
    if type(title) == str:
        plt.savefig(f'mag_variation_cmaes/' + title + '.png')
    else:
        plt.savefig(f'mag_variation_cmaes/step_plot_total_correction_differences_energy_pixel{pixel1}_{year}_{month}_{day}_multiple_offsets.png')
    plt.close('all')

In [ ]:
opt_b_ts = [array([-0.51750594,  1.0390966 ,  2.95802849]), array([ 0.75955021,  0.76883405, -0.10316687]), array([-1.51450692,  1.15373077, -2.02308867]), array([-3.50551777, -0.7891109 ,  4.23075418]), array([-1.88151883, -0.62277722, -2.11933347]), array([-0.21840517, -1.58434976,  0.57044087]), array([-0.27608079, -1.5068718 ,  0.43600885]), array([ 0.16117981, -1.36556997,  0.77313155])]

step_plot_ideal_offsets_each_ts(dat, period, grenz, opt_b_ts, title='ideal_mag_offsets_ts_2021_12_04)